In [1]:
from clean_asurv import read_asurv

In [2]:
years_through_2011 = 30
temporal_res = 'weekly'
box_length_m = 500
complete_idx_square = True
keep_geometry_col = False
# END PARAMS

asurv_gdf = read_asurv(years_through_2011, temporal_res=temporal_res)

## x, y, time, count

In [3]:
asurv_gdf.columns

Index(['Year', 'Month', 'Day', 'Survey_year', 'Coordinates', 'X', 'Y', 'WHITE',
       'JUVE', 'UNK', 'Band', 'Band_condition', 'Observer', 'Pilot',
       'Aircraft', 'Area', 'Land_Cover', 'Time', 'Weather', 'Temperature',
       'Comments', 'geometry', 'date', 'week', 'week_name', 'counts'],
      dtype='object')

In [6]:
asurv_gdf[['date', 'week', 'week_name', 'X', 'Y', 'counts', 'geometry']]

,date,week,week_name,X,Y,counts,geometry
9894,1979-04-13,15,1979-04-08_to_1979-04-15,711116.3135,3.120143e+06,3.0,POINT (711116.314 3120142.649)
9895,1979-04-13,15,1979-04-08_to_1979-04-15,710976.4147,3.118770e+06,2.0,POINT (710976.415 3118770.451)
9896,1979-04-13,15,1979-04-08_to_1979-04-15,708294.0421,3.118293e+06,3.0,POINT (708294.042 3118292.944)
9897,1979-04-13,15,1979-04-08_to_1979-04-15,703759.2864,3.113870e+06,2.0,POINT (703759.286 3113869.949)
9898,1979-04-13,15,1979-04-08_to_1979-04-15,703378.2856,3.113591e+06,5.0,POINT (703378.286 3113590.548)
...,...,...,...,...,...,...,...
38327,2011-03-01,1679,2011-02-27_to_2011-03-06,715558.6950,3.124125e+06,2.0,POINT (715558.695 3124124.849)
38328,2011-03-01,1679,2011-02-27_to_2011-03-06,713971.1918,3.123273e+06,2.0,POINT (713971.192 3123272.889)
38329,2011-03-01,1679,2011-02-27_to_2011-03-06,713637.8162,3.122786e+06,3.0,POINT (713637.816 3122786.054)
38330,2011-03-01,1679,2011-02-27_to_2011-03-06,712955.1898,3.121251e+06,2.0,POINT (712955.19 3121251.468)
